In [1]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pandas as pd
import pickle

In [2]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

ddbthr = 0.64

In [3]:
year = '2018'
outsum = processor.dict_accumulator()

In [4]:
# Check if pickle exists, and don't re-create it if it does
picklename = year+'/templates.pkl'
templates = pickle.load(open(picklename,'rb')).sum('pt1','msd1').integrate('systematic','nominal').integrate('mjj',overflow='allnan')
templates

<Hist (process,region,genflavor,ddb1) instance at 0x7f892fa8e3a0>

In [5]:
mc = ['QCD','Wjets','Zjets','Zjetsbb','VV','ttbar','singlet','ggF','VBF','WH','ZH','ttH']
cutflow = pd.DataFrame(columns=['ggffail','ggfpass','vbffail','vbfpass'],index=mc)

In [6]:
for p in mc:
    if "Zjets" in p:
        continue
    
    cutflow['ggffail'][p] = templates.sum('genflavor').integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-ggf').integrate('process',p).values()[()]
    cutflow['ggfpass'][p] = templates.sum('genflavor').integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-ggf').integrate('process',p).values()[()]
    cutflow['vbffail'][p] = templates.sum('genflavor').integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-vbf').integrate('process',p).values()[()]
    cutflow['vbfpass'][p] = templates.sum('genflavor').integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-vbf').integrate('process',p).values()[()]
    
cutflow['ggffail']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['ggfpass']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['vbffail']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]
cutflow['vbfpass']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]

cutflow['ggffail']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['ggfpass']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['vbffail']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]
cutflow['vbfpass']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]

In [7]:
cutflow.astype('int')

,ggffail,ggfpass,vbffail,vbfpass
QCD,6447758,15718,190531,554
Wjets,142488,367,4386,11
Zjets,53956,267,1779,9
Zjetsbb,6953,2262,206,75
VV,2101,47,53,1
ttbar,29431,349,984,10
singlet,3386,34,111,1
ggF,123,48,7,2
VBF,36,15,23,11
WH,30,9,0,0


In [8]:
(cutflow['ggffail']['ggF']+cutflow['ggfpass']['ggF']+cutflow['ggffail']['VBF']+cutflow['ggfpass']['VBF'])/sum(cutflow['ggffail']+cutflow['ggfpass'])

3.347767973986329e-05

In [9]:
(cutflow['vbffail']['ggF']+cutflow['vbfpass']['ggF']+cutflow['vbffail']['VBF']+cutflow['vbfpass']['VBF'])/sum(cutflow['vbffail']+cutflow['vbfpass'])

0.0002256464989344803

In [10]:
cutflow.astype('int').to_latex(buf=year+'/cutflow-final.tex')

/tmp/ipykernel_31231/3630590848.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  cutflow.astype('int').to_latex(buf=year+'/cutflow-final.tex')
